## Loading saved models

- Example code for loading saved models generated and saved by "full dataset ML.ipynb"
- input: raw spectral data 
- output: 4 predicted values

In [100]:
## load libraries
import math
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from keras.models import Sequential
from keras.layers import Dense
import keras.regularizers as kr
from keras.models import load_model

In [1]:
def avg_over_col(df, num_col):
    new_df = pd.DataFrame()
    curr_col = 0
    for i in range(num_col,len(df.columns)+1, num_col):
        working_df = df.iloc[:,curr_col:i]
        colname = str(working_df.columns[0])
        new_df[colname]=working_df.mean(axis = 1)
        curr_col += num_col
    return(new_df)

In [89]:
# read file, output values averaged over 4 wavelengths, wide format
def read_pre_process(filename): 
    file = pd.read_table(filename, header = None, index_col = None )
    file.columns = ['to_del','Wavelength', 'Intensity']
    file = file.drop('to_del', axis = 1)
    file['Wavelength']=file['Wavelength'].apply(lambda x: math.floor(x))
    file = file.groupby(['Wavelength']).agg(np.mean).reset_index()
    file = file[file['Wavelength'].between(450, 850)].set_index('Wavelength')
    return file.transpose()

In [92]:
# normalization and standardization models
norms = joblib.load('normalizer.save')
stds = joblib.load('standarizer.save')

In [90]:
# read anad preprocess data
## input: data with headers as wavelengths

filename = 'test.txt'
df = read_pre_process(filename)

In [96]:
df_norm = avg_over_col(pd.DataFrame(norms.transform(df), columns = df.columns),4)
df_std = avg_over_col(pd.DataFrame(stds.transform(df), columns = df.columns),4)

In [101]:
# load keras models 
sigmoid_model_n = load_model('sigmoid_norm.h5')
sigmoid_L1_model_n =load_model('sigmoid_L1_norm.h5')
sigmoid_model_s = load_model('sigmoid_std.h5')
sigmoid_L1_model_s = load_model('sigmoid_L1_std.h5')

In [102]:
sigmoid_model_n.predict(df_norm)

array([[ 3.88969898]], dtype=float32)

In [104]:
sigmoid_L1_model_n.predict(df_norm)

array([[ 3.86917067]], dtype=float32)

In [105]:
sigmoid_model_s.predict(df_std)

array([[ 4.10934877]], dtype=float32)

In [106]:
sigmoid_L1_model_s.predict(df_std)

array([[ 4.14122009]], dtype=float32)